<a href="https://colab.research.google.com/github/HkrAnis/Python-SDV-SyntheticData-ML/blob/main/220614_Hackathon_Task1_Gaussion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install SDV (Used for the synthesizer)

In [ ]:
pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import packages

In [ ]:
from google.colab import drive
from sdv.demo import load_tabular_demo
from sdv.tabular import GaussianCopula
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Connect to the goolge drive dataset

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import the dataset from the google drive 

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Hackathon/data_with_missing.csv")
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,ID,Race,Education,tag_truncated,year,country
0,0,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,20000.0,10000.0,Unavailable,7392,1,Unavailable,Unavailable,NaN,2017-01-01,US
1,1,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7419,2,Unavailable,Unavailable,NaN,2017-01-01,US
2,2,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,0.0,0.0,Unavailable,11527,3,Unavailable,Unavailable,NaN,2017-01-01,US
3,3,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,180000.0,35000.0,Unavailable,7472,7,Unavailable,Unavailable,NaN,2017-01-01,US
4,4,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7322,9,Unavailable,Unavailable,NaN,2017-01-01,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,35319,Amazon,L5,Software Engineer,213000,"Seattle, WA",6.0,4.0,Distributed Systems (Back-End),147500.0,38000.0,35000.0,Unavailable,11527,1843,Unavailable,Unavailable,Distributed Systems (Back-End),2018-01-01,US
35320,35320,eBay,Designer III,Product Designer,172000,"San Jose, CA",4.0,2.0,Design,142000.0,15000.0,14200.0,Unavailable,7422,1958,Unavailable,Unavailable,others,2018-01-01,US
35321,35321,Google,T4,Software Engineer,327000,"Seattle, WA",10.0,1.0,Distributed Systems (Back-End),155000.0,150000.0,22000.0,Unavailable,11527,1973,Unavailable,Unavailable,Distributed Systems (Back-End),2018-01-01,US
35322,35322,Microsoft,62,Software Engineer,237000,"Redmond, WA",2.0,2.0,Full Stack,146900.0,73200.0,16000.0,Unavailable,11521,2037,Unavailable,Unavailable,Full Stack,2018-01-01,US


### Remove Unnamed 0 column 

In [ ]:
df.drop(columns=["Unnamed: 0", "tag_truncated", "totalyearlycompensation", "ID"], inplace=True)

### Drop the row from city_id where anonymized is present

In [ ]:
i = df[(df.cityid == 'anonymized')].index
df.drop(i, inplace=True)
df

,company,level,title,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,Race,Education,year,country
0,Oracle,L3,Product Manager,"Redwood City, CA",1.5,1.5,NaN,107000.0,20000.0,10000.0,Unavailable,7392,Unavailable,Unavailable,2017-01-01,US
1,eBay,SE 2,Software Engineer,"San Francisco, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7419,Unavailable,Unavailable,2017-01-01,US
2,Amazon,L7,Product Manager,"Seattle, WA",8.0,0.0,NaN,155000.0,0.0,0.0,Unavailable,11527,Unavailable,Unavailable,2017-01-01,US
3,Apple,M1,Software Engineering Manager,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,180000.0,35000.0,Unavailable,7472,Unavailable,Unavailable,2017-01-01,US
4,Microsoft,60,Software Engineer,"Mountain View, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7322,Unavailable,Unavailable,2017-01-01,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,Amazon,L5,Software Engineer,"Seattle, WA",6.0,4.0,Distributed Systems (Back-End),147500.0,38000.0,35000.0,Unavailable,11527,Unavailable,Unavailable,2018-01-01,US
35320,eBay,Designer III,Product Designer,"San Jose, CA",4.0,2.0,Design,142000.0,15000.0,14200.0,Unavailable,7422,Unavailable,Unavailable,2018-01-01,US
35321,Google,T4,Software Engineer,"Seattle, WA",10.0,1.0,Distributed Systems (Back-End),155000.0,150000.0,22000.0,Unavailable,11527,Unavailable,Unavailable,2018-01-01,US
35322,Microsoft,62,Software Engineer,"Redmond, WA",2.0,2.0,Full Stack,146900.0,73200.0,16000.0,Unavailable,11521,Unavailable,Unavailable,2018-01-01,US


### Replace all the values where tags are less than 100 with Other

In [ ]:
tag= df["tag"].value_counts()
df["tag"]= np.where(df["tag"].isin(tag.index[tag>100]), df["tag"], "Other")
df["tag"].value_counts()

Distributed Systems (Back-End)    6745
Other                             6023
Full Stack                        5788
API Development (Back-End)        3281
ML / AI                           2937
Web Development (Front-End)       1458
Product                           1048
iOS                                791
DevOps                             743
Data                               737
Security                           721
Networking                         683
Android                            604
Mobile (iOS + Android)             563
Technical                          479
Testing (SDET)                     409
User Experience (UX)               395
Site Reliability (SRE)             360
General                            352
Production                         205
Operations                         189
Consumer                           182
Enterprise                         182
Hardware                           163
ASIC Design                        149
Cloud                    

### Replace all the values where levels are less than 100 with Other

In [ ]:
level= df["level"].value_counts()
df["level"]= np.where(df["level"].isin(level.index[level>100]), df["level"], "Other")
df["level"].value_counts()

Other     7720
L5        3432
L4        3201
L6        2080
L3        1578
          ... 
Band 7     103
4          103
6          103
5          102
L5A        101
Name: level, Length: 69, dtype: int64

### Unmount from google drive

In [ ]:
# drive.flush_and_unmount()

### Define the function that will prepare the model for ML

In [ ]:
def modelPreperation(dataSet):
  dataSet.dropna(inplace=True)
  dataSet=pd.get_dummies(dataSet, columns=["title","company", "level", "location", "tag", "gender", "Race", "Education", "country"])
  dataSet["year"]=dataSet["year"].str[:4].astype(int)
  return dataSet

### Define a function that would Split the dataset between X and y 

In [ ]:
def splitter(df, Ycol):
    # This function takes a dataset and splits it between the independent columns and the dependent column
    # df is a pandas dataframe
    # Ycol is a string representing the column to split 
    return df.loc[:, df.columns != Ycol], df[Ycol]

### Divide the dataset between train and test

In [ ]:
train = df.sample(frac = 0.8, random_state=7)
test = df.drop(train.index)

### Create the synthesize train set 

1-Define the metadata

In [ ]:
Metadata= {'fields':
           {'company': {'type': 'categorical'},
            'level': {'type': 'categorical'},
            'title': {'type': 'categorical'},
            'totalyearlycompensation': {'type': 'numerical', 'subtype': 'float'},
            'location': {'type': 'categorical'},
            'yearsofexperience': {'type': 'numerical', 'subtype': 'float'},
            'yearsatcompany': {'type': 'numerical', 'subtype': 'float'},
            'tag': {'type': 'categorical'},
            'basesalary': {'type': 'numerical', 'subtype': 'float'},
            'stockgrantvalue': {'type': 'numerical', 'subtype': 'float'},
            'bonus': {'type': 'numerical', 'subtype': 'float'},
            'gender': {'type': 'categorical'},
            'cityid': {'type': 'categorical'},
            'ID': {'type': 'categorical'},
            'Race': {'type': 'categorical'},
            'Education': {'type': 'categorical'},
            'year': {'type': 'categorical'},
            'country': {'type': 'categorical'}
            },
           
           'constraints': [],
           "primary_key": []}

2- Choose the synthesizer model

In [ ]:
model = GaussianCopula()

3- Fit the chosen model 

In [ ]:
model.fit(train)

/usr/local/lib/python3.7/dist-packages/copulas/univariate/truncated_gaussian.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  a = (self.min - loc) / scale
/usr/local/lib/python3.7/dist-packages/copulas/univariate/truncated_gaussian.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  b = (self.max - loc) / scale


4- Save the model

In [ ]:
model.save('my_model.pkl')

5- Load the model 

In [ ]:
loaded = GaussianCopula.load('my_model.pkl')

6- Create the synthetic dataset

In [ ]:
train_s = loaded.sample(num_rows=len(train))
train_s

,company,level,title,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,Race,Education,year,country
0,Amazon,E5,Software Engineer,"Seattle, WA",3.83,4.529,ML / AI,159577.252949,65096.66,2405.68,Male,11521,Unavailable,PhD,2021-01-01,US
1,Amazon,L3,Product Manager,"Mountain View, CA",5.46,6.029,Distributed Systems (Back-End),134535.226207,15518.07,17373.09,Male,7434,Unavailable,Unavailable,2021-01-01,US
2,Apple,Other,Software Engineer,"Santa Clara, CA",5.44,1.364,API Development (Back-End),179626.062132,104396.10,7632.80,Female,7839,Asian,PhD,2020-01-01,Canada
3,Expedia,Staff Software Engineer,Software Engineer,"San Francisco, CA",10.78,1.054,Distributed Systems (Back-End),96834.806418,45161.64,37611.48,Unavailable,7419,Unavailable,Unavailable,2020-01-01,US
4,Amazon,Senior Software Engineer,Software Engineer,"New York, NY",3.47,2.708,Data,82343.117727,82929.68,22998.80,Male,10182,Unavailable,Master's Degree,2021-01-01,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28253,Qualcomm,L5,Data Scientist,"Austin, TX",3.39,2.250,Data,174241.713783,143375.21,7621.85,Male,7416,Unavailable,Unavailable,2020-01-01,US
28254,Oracle,62,Software Engineer,"Sunnyvale, CA",13.66,3.909,Other,205587.206681,135797.62,6292.56,Unavailable,7322,Unavailable,Unavailable,2021-01-01,US
28255,Nutanix,Software Engineer,Software Engineer,"Austin, TX",8.79,3.898,Networking,58752.287387,47738.32,7862.36,Female,7434,Unavailable,Unavailable,2017-01-01,US
28256,Microsoft,L6,Product Manager,"San Francisco, CA",1.25,2.366,Other,137995.355704,24532.76,35238.11,Unavailable,7300,Unavailable,Unavailable,2019-01-01,US


### Append train_s and df

1- Create a new column in train_s and df to be able to seperate them later on 

In [ ]:
df["Table_number"]=0
train_s["Table_number"]=1 
print(df.columns, train_s.columns)

Index(['company', 'level', 'title', 'location', 'yearsofexperience',
       'yearsatcompany', 'tag', 'basesalary', 'stockgrantvalue', 'bonus',
       'gender', 'cityid', 'Race', 'Education', 'year', 'country',
       'Table_number'],
      dtype='object') Index(['company', 'level', 'title', 'location', 'yearsofexperience',
       'yearsatcompany', 'tag', 'basesalary', 'stockgrantvalue', 'bonus',
       'gender', 'cityid', 'Race', 'Education', 'year', 'country',
       'Table_number'],
      dtype='object')


2- Append train_s and df 

In [ ]:
df_appended= df.append(train_s, ignore_index=True)
df_appended

,company,level,title,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,Race,Education,year,country,Table_number
0,Oracle,L3,Product Manager,"Redwood City, CA",1.50,1.500,Other,107000.000000,20000.00,10000.00,Unavailable,7392,Unavailable,Unavailable,2017-01-01,US,0
1,eBay,Other,Software Engineer,"San Francisco, CA",5.00,3.000,Other,0.000000,0.00,0.00,Unavailable,7419,Unavailable,Unavailable,2017-01-01,US,0
2,Amazon,L7,Product Manager,"Seattle, WA",8.00,0.000,Other,155000.000000,0.00,0.00,Unavailable,11527,Unavailable,Unavailable,2017-01-01,US,0
3,Apple,M1,Software Engineering Manager,"Sunnyvale, CA",7.00,5.000,Other,157000.000000,180000.00,35000.00,Unavailable,7472,Unavailable,Unavailable,2017-01-01,US,0
4,Microsoft,60,Software Engineer,"Mountain View, CA",5.00,3.000,Other,0.000000,0.00,0.00,Unavailable,7322,Unavailable,Unavailable,2017-01-01,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63576,Qualcomm,L5,Data Scientist,"Austin, TX",3.39,2.250,Data,174241.713783,143375.21,7621.85,Male,7416,Unavailable,Unavailable,2020-01-01,US,1
63577,Oracle,62,Software Engineer,"Sunnyvale, CA",13.66,3.909,Other,205587.206681,135797.62,6292.56,Unavailable,7322,Unavailable,Unavailable,2021-01-01,US,1
63578,Nutanix,Software Engineer,Software Engineer,"Austin, TX",8.79,3.898,Networking,58752.287387,47738.32,7862.36,Female,7434,Unavailable,Unavailable,2017-01-01,US,1
63579,Microsoft,L6,Product Manager,"San Francisco, CA",1.25,2.366,Other,137995.355704,24532.76,35238.11,Unavailable,7300,Unavailable,Unavailable,2019-01-01,US,1


3- Prepare the dataframe for the machine learning model

In [ ]:
df_appended= modelPreperation(df_appended)
df_appended

,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,cityid,year,Table_number,title_Business Analyst,title_Data Scientist,...,Education_Bachelor's Degree,Education_Highschool,Education_Master's Degree,Education_PhD,Education_Some College,Education_Unavailable,country_Canada,country_Germany,country_US,country_United Kingdom
0,1.50,1.500,107000.000000,20000.00,10000.00,7392,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,5.00,3.000,0.000000,0.00,0.00,7419,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,8.00,0.000,155000.000000,0.00,0.00,11527,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,7.00,5.000,157000.000000,180000.00,35000.00,7472,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,5.00,3.000,0.000000,0.00,0.00,7322,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63576,3.39,2.250,174241.713783,143375.21,7621.85,7416,2020,1,0,1,...,0,0,0,0,0,1,0,0,1,0
63577,13.66,3.909,205587.206681,135797.62,6292.56,7322,2021,1,0,0,...,0,0,0,0,0,1,0,0,1,0
63578,8.79,3.898,58752.287387,47738.32,7862.36,7434,2017,1,0,0,...,0,0,0,0,0,1,0,0,1,0
63579,1.25,2.366,137995.355704,24532.76,35238.11,7300,2019,1,0,0,...,0,0,0,0,0,1,0,0,1,0


4- Split the dataset between train_s and df

In [ ]:
df= df_appended[df_appended["Table_number"]==0]
df

,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,cityid,year,Table_number,title_Business Analyst,title_Data Scientist,...,Education_Bachelor's Degree,Education_Highschool,Education_Master's Degree,Education_PhD,Education_Some College,Education_Unavailable,country_Canada,country_Germany,country_US,country_United Kingdom
0,1.5,1.5,107000.0,20000.0,10000.0,7392,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,5.0,3.0,0.0,0.0,0.0,7419,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,8.0,0.0,155000.0,0.0,0.0,11527,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,7.0,5.0,157000.0,180000.0,35000.0,7472,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,5.0,3.0,0.0,0.0,0.0,7322,2017,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35318,6.0,4.0,147500.0,38000.0,35000.0,11527,2018,0,0,0,...,0,0,0,0,0,1,0,0,1,0
35319,4.0,2.0,142000.0,15000.0,14200.0,7422,2018,0,0,0,...,0,0,0,0,0,1,0,0,1,0
35320,10.0,1.0,155000.0,150000.0,22000.0,11527,2018,0,0,0,...,0,0,0,0,0,1,0,0,1,0
35321,2.0,2.0,146900.0,73200.0,16000.0,11521,2018,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
train_s= df_appended[df_appended["Table_number"]==1]
train_s

,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,cityid,year,Table_number,title_Business Analyst,title_Data Scientist,...,Education_Bachelor's Degree,Education_Highschool,Education_Master's Degree,Education_PhD,Education_Some College,Education_Unavailable,country_Canada,country_Germany,country_US,country_United Kingdom
35323,3.83,4.529,159577.252949,65096.66,2405.68,11521,2021,1,0,0,...,0,0,0,1,0,0,0,0,1,0
35324,5.46,6.029,134535.226207,15518.07,17373.09,7434,2021,1,0,0,...,0,0,0,0,0,1,0,0,1,0
35325,5.44,1.364,179626.062132,104396.10,7632.80,7839,2020,1,0,0,...,0,0,0,1,0,0,1,0,0,0
35326,10.78,1.054,96834.806418,45161.64,37611.48,7419,2020,1,0,0,...,0,0,0,0,0,1,0,0,1,0
35327,3.47,2.708,82343.117727,82929.68,22998.80,10182,2021,1,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63576,3.39,2.250,174241.713783,143375.21,7621.85,7416,2020,1,0,1,...,0,0,0,0,0,1,0,0,1,0
63577,13.66,3.909,205587.206681,135797.62,6292.56,7322,2021,1,0,0,...,0,0,0,0,0,1,0,0,1,0
63578,8.79,3.898,58752.287387,47738.32,7862.36,7434,2017,1,0,0,...,0,0,0,0,0,1,0,0,1,0
63579,1.25,2.366,137995.355704,24532.76,35238.11,7300,2019,1,0,0,...,0,0,0,0,0,1,0,0,1,0


5- Remove the table number column

In [ ]:
df.drop(columns=["Table_number"], inplace=True)
train_s.drop(columns=["Table_number"], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Split df between the train part and the test part

In [ ]:
train = df.sample(frac = 0.8, random_state=7)
test = df.drop(train.index)

### Split train, train_s and test

*   X_train and y_train
*   X_train_s and y_train_s
*   X_test and y_test

In [ ]:
X_train, y_train= splitter(train, "basesalary")
X_train_s, y_train_s= splitter(train_s, "basesalary")
X_test, y_test= splitter(test, "basesalary")

### Score the diffent machine learning models

1- Create a list and assign a random seed

In [ ]:
pipeline=[]
seed=7

2- Create the machine learning models that are going to be used 

In [ ]:
Lasso = Pipeline([('scaler', StandardScaler()), ('Lasso', ElasticNet(random_state=seed))])
Ridge = Pipeline([('scaler', StandardScaler()), ('Ridge', Ridge(random_state=seed, tol=10))])
ExtraTreesRegressor = Pipeline([("Scaler", StandardScaler()),("ET", ExtraTreesRegressor(random_state=seed))])
BaggingRegressor = Pipeline([("Scaler", StandardScaler()),("BR", BaggingRegressor(random_state=seed))])
HuberRegressor = Pipeline([("Scaler", StandardScaler()),("Hub-Reg", HuberRegressor())])
BayesianRidge = Pipeline([("Scaler", StandardScaler()),("BR", BayesianRidge())])
DecisionTreeRegressor = Pipeline([("Scaler", StandardScaler()),("DT_reg", DecisionTreeRegressor())])
KNeighborsRegressor = Pipeline([("Scaler", StandardScaler()),("KNN_reg", KNeighborsRegressor())])
GradientBoostingRegressor =  Pipeline([("Scaler", StandardScaler()),("GBoost-Reg", GradientBoostingRegressor())])
                 


3- Append the list with the models and their names

In [ ]:
pipeline.append(("Lasso",Lasso))
pipeline.append(("Ridge",Ridge))
pipeline.append(("ExtraTreesRegressor",ExtraTreesRegressor)) 
pipeline.append(("BaggingRegressor",BaggingRegressor)) 
pipeline.append(("HuberRegressor",HuberRegressor)) 
pipeline.append(("BayesianRidge",BayesianRidge)) 
pipeline.append(("DecisionTreeRegressor",DecisionTreeRegressor)) 
pipeline.append(("KNeighborsRegressor",KNeighborsRegressor)) 
pipeline.append(("GradientBoostingRegressor",GradientBoostingRegressor)) 

4- Define the parameters of the cross validation

In [ ]:
n_folds = 3
kfold = KFold(n_splits=n_folds, random_state=seed,shuffle=True)

### Print the cross validated results of:


1.   Cross validated train 
2.   Cross validated train_s
3.   train on test 
4.   train_s on test








In [ ]:
pipeline

[('Lasso', Pipeline(steps=[('scaler', StandardScaler()),
                  ('Lasso', ElasticNet(random_state=7))])),
 ('Ridge', Pipeline(steps=[('scaler', StandardScaler()),
                  ('Ridge', Ridge(random_state=7, tol=10))])),
 ('ExtraTreesRegressor', Pipeline(steps=[('Scaler', StandardScaler()),
                  ('ET', ExtraTreesRegressor(random_state=7))])),
 ('BaggingRegressor', Pipeline(steps=[('Scaler', StandardScaler()),
                  ('BR', BaggingRegressor(random_state=7))])),
 ('HuberRegressor',
  Pipeline(steps=[('Scaler', StandardScaler()), ('Hub-Reg', HuberRegressor())])),
 ('BayesianRidge',
  Pipeline(steps=[('Scaler', StandardScaler()), ('BR', BayesianRidge())])),
 ('DecisionTreeRegressor', Pipeline(steps=[('Scaler', StandardScaler()),
                  ('DT_reg', DecisionTreeRegressor())])),
 ('KNeighborsRegressor', Pipeline(steps=[('Scaler', StandardScaler()),
                  ('KNN_reg', KNeighborsRegressor())])),
 ('GradientBoostingRegressor', Pipeline

In [ ]:
list_df=[]
for name, model in pipeline:

    # Train score mean 
    cv_results_train = cross_val_score(model, X_train, y_train, cv= kfold, scoring='r2')
    Train_score_mean= cv_results_train.mean()
    # Train score standard deviation 
    Train_score_std= cv_results_train.std()
    # Test score 
    model.fit(X_train, y_train)
    Test= model.score(X_test, y_test)
    
    # Train_s score mean
    cv_results_train_s = cross_val_score(model, X_train_s, y_train_s, cv= kfold, scoring='r2')
    Train_s_score_mean= cv_results_train_s.mean()
    # Train_s standard deviation
    Train_s_score_std= cv_results_train_s.std()
    # Test_s score 
    model.fit(X_train_s, y_train_s)
    Test_s= model.score(X_test, y_test)

    list_df.append([name,Train_score_mean, Train_score_std, Test, Train_s_score_mean, Train_s_score_std, Test_s])

In [ ]:
Scores=pd.DataFrame(list_df, columns=["Model_Name", "Train_score_Mean", "Train_score_STD", "Train_score_Test", "Train_synthetic_Mean", "Train_synthetic_STD", "Train_synthetic_score_Test"])

In [ ]:
Scores

,Model_Name,Train_score_Mean,Train_score_STD,Train_score_Test,Train_synthetic_Mean,Train_synthetic_STD,Train_synthetic_score_Test
0,Lasso,0.533155,0.019724,0.536689,0.436289,0.001041,0.409485
1,Ridge,0.567567,0.016362,0.566523,0.473747,0.000737,0.281343
2,ExtraTreesRegressor,0.820528,0.026534,0.835823,0.473527,0.004066,0.343718
3,BaggingRegressor,0.800616,0.025991,0.836656,0.430161,0.003419,0.234324
4,HuberRegressor,0.539811,0.016063,0.532523,0.472617,0.001380,0.273188
5,BayesianRidge,0.567832,0.016716,0.566783,0.473864,0.000747,0.287865
6,DecisionTreeRegressor,0.687723,0.030013,0.720641,-0.009714,0.007251,-0.293391
7,KNeighborsRegressor,0.355895,0.010556,0.385949,-0.043709,0.018566,0.057762
8,GradientBoostingRegressor,0.775161,0.026946,0.783380,0.503305,0.002893,0.334766
